# Simple Recommender Models

In [ ]:
!pip install -q implicit

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix
from sklearn.model_selection import GridSearchCV

from implicit.nearest_neighbours import *
from implicit.evaluation import * 
from implicit.evaluation import train_test_split

from metrics import precision_at_k, recall_at_k

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_parquet('./data/bronze/train.parquet.gzip')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.shape, data_test.shape

((2278490, 12), (118314, 12))

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [ ]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))
print('There are {} users in test set out of which {} are new.'.format(test_users, new_test_users))

There are 2042 users in test set out of which 0 are new.


## Random recommender

In [ ]:
def random_recommendation(items, n=5):
    """randomly picks n items from given list without replacement"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [ ]:
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[926804, 1066866, 13380924, 12582401, 14111641]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1914142, 2534864, 1592534, 5995492, 9705975]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15454651, 12485163, 14111053, 930504, 1038963]"


## Weighted random recommender

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """picks n items from given list in proportion of their weights"""

    recs = np.random.choice(item_weights['item_id'], p=item_weights['weight'], size=n, replace=False)
    
    return recs.tolist()

In [ ]:
# weight for item is equivalent to the popularity of that item
# popularity is measured by the percentage purchase of that item
item_weights = data.groupby('item_id')['user_id'].count().reset_index()
item_weights.sort_values('user_id', ascending=False, inplace=True)
total_count = item_weights['user_id'].sum()
item_weights['weight'] = item_weights['user_id'] / total_count
item_weights.head()

,item_id,user_id,weight
35054,1082185,27362,0.011416
56233,6534178,18364,0.007662
29195,1029743,13455,0.005614
25333,995242,11397,0.004755
37719,1106523,9175,0.003828


In [ ]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))
result.head()

,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[926804, 1066866, 13380924, 12582401, 14111641]","[854133, 1039578, 1082185, 1132178, 900370]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1914142, 2534864, 1592534, 5995492, 9705975]","[874736, 1065303, 5573052, 1198103, 5592931]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15454651, 12485163, 14111053, 930504, 1038963]","[1163514, 1134398, 834117, 1038038, 1082185]"


## Popularity-based recommender

In [ ]:
def popularity_recommendation(data, n=5):
    """Top-n popular items by their sales values"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [ ]:
popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head()

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[926804, 1066866, 13380924, 12582401, 14111641]","[854133, 1039578, 1082185, 1132178, 900370]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1914142, 2534864, 1592534, 5995492, 9705975]","[874736, 1065303, 5573052, 1198103, 5592931]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15454651, 12485163, 14111053, 930504, 1038963]","[1163514, 1134398, 834117, 1038038, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]"


## Item-item recommender

In [ ]:
item_qty = data_train.groupby('item_id')['quantity'].sum().reset_index()
item_qty.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_qty.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [ ]:
item_qty_top5k = item_qty.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
# Let's create a fictitious item_id (if the user bought goods from the top 5000, then he "bought" such a product)
data_train.loc[ ~ data_train['item_id'].isin(item_qty_top5k), 'item_id'] = 6666
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix[user_item_matrix > 0] = 1
user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
sparse_user_item

<2499x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 667080 stored elements in Compressed Sparse Row format>

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
model = ItemItemRecommender(K=5, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
recs = model.recommend(userid=userid_to_id[2],
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5,
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [ ]:
result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]"


## Cosine recommender

In [ ]:
model = CosineRecommender(K=5, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]"


## TF-IDF recommender

In [ ]:
model = TFIDFRecommender(K=5, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

In [ ]:
result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"


## Item-item recommender with filtering + (K=1)

In [ ]:
model = ItemItemRecommender(K=1, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9365874, 10204591, 1082641, 873588, 9297493]","[992650, 874972, 995242, 882830, 1029743]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1109948, 1109459, 15717068, 958027, 884344]","[1098066, 861445, 1095275, 15626040, 1133018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]"


## Comparison

In [ ]:
results_precision_at_k = {}
results_recall_at_k = {}

for name_col in result.columns[1:]:
    results_precision_at_k[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)
    results_recall_at_k[name_col] = round(result.apply(lambda row: recall_at_k(row[name_col], row['actual']), axis=1).mean(),4)

In [ ]:
results_precision_at_k = pd.DataFrame.from_dict(results_precision_at_k, orient='index', columns=['precision_at_k'])
results_recall_at_k = pd.DataFrame.from_dict(results_recall_at_k, orient='index', columns=['recall_at_k'])
pd.concat([results_precision_at_k, results_recall_at_k], axis=1).sort_values(by='precision_at_k', ascending=False)

,precision_at_k,recall_at_k
actual,1.0000,0.2710
own_purchases,0.2199,0.0289
popular_recommendation,0.1552,0.0250
tfidf,0.1390,0.0154
itemitem,0.1368,0.0157
cosine,0.1329,0.0148
weighted_random_recommendation,0.0176,0.0020
random_recommendation,0.0011,0.0001
